<a href="https://colab.research.google.com/github/CristinaMarsh/Data/blob/Image/XarrayCan_be_directly_opened_used_to_obtain_remote_sensing_data_and_reanalysis_data_Cloud_Optimized_GeoTIFF_(COG)_with_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! git clone https://github.com/sat-utils/sat-search.git
% cd sat-search
! pip install .

Cloning into 'sat-search'...
remote: Enumerating objects: 1816, done.
remote: Counting objects: 100% (125/125), done.
remote: Compressing objects: 100% (80/80), done.
remote: Total 1816 (delta 73), reused 85 (delta 41), pack-reused 1691
Receiving objects: 100% (1816/1816), 351.67 KiB | 2.96 MiB/s, done.
Resolving deltas: 100% (1123/1123), done.
/content/sat-search
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/sat-search
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
     |████████████████████████████████| 225 kB 7.0 MB/s 
  Created wheel for sat-search: filename=sat_searc

In [ ]:
! pip install rasterio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.3 MB 4.8 MB/s 


In [ ]:
from satsearch import Search
import rasterio as rio
from rasterio.plot import show
import matplotlib.pyplot as plt

## Simple Query

Satsearch can take bounding boxes or Geojsons like Polygon. So let us start with a simple bounding box. We are using the Element84 free sentinel 2 data STAC API available in Amazon for free, but you can use any public STAC API endpoint.

In [ ]:
#https://www.element84.com/earth-search/
bbox = [ 11.756057739257812,
          57.649809962218995,
          12.10693359375,
          57.751442372568924
       ]
url = 'https://earth-search.aws.element84.com/v0'
bbox_search = Search(bbox=bbox,  url=url)
print('Query returned {} items'.format(bbox_search.found()))

Query returned 8159 items


With the Search method from satsearch, we can find out how many items are available in the bounding box.

You can limit the data with other parameters like dates and cloud coverage. So let us add these parameters to the mix.

In [ ]:
bbox_search = Search(bbox=bbox, datetime="2021-01-01/2021-02-01", query={'eo:cloud_cover': {'lt': 5}}, url=url)
print('Query returned {} items'.format(bbox_search.found()))

Query returned 12 items


The above code will return images between January to February 2021, with less than 5% cloud coverage.

You can also bring in a bounding box from the Geopandas Geometry column or pass here a polygon coordinates like this, and it will work like the above example.

In [ ]:
polygon = {
    "type": "Polygon",
    "coordinates": [
        [
            [11.756057739257812, 57.649809962218995],
            [12.10693359375, 57.649809962218995],
            [12.10693359375, 57.751442372568924],
            [11.756057739257812, 57.751442372568924],
            [11.756057739257812, 57.649809962218995],
        ]
    ],
}
intersect_search = Search(intersects=polygon, url=url)
print('Query returned {} items'.format(bbox_search.found()))

Query returned 12 items


And we get the same results from the STAC catalog search.

STAC Results
Once you search and find out that some images are available for the given dates, you can fetch the results. Let us have more specific query parameters to filter out the returned number of scenes.

In [ ]:
bbox_search = Search(
    bbox=bbox,
    datetime="2020-10-01/2021-02-01",
    query={"eo:cloud_cover": {"lt": 1}},
    collections=["sentinel-s2-l2a-cogs"],
    url=url,
)
items = bbox_search.items()
print(items.summary())

Items (9):
date                      id                        
2020-12-25                S2B_32VPK_20201225_0_L2A  
2020-11-30                S2A_32VPJ_20201130_0_L2A  
2020-11-30                S2A_33VUD_20201130_0_L2A  
2020-11-30                S2A_32VPK_20201130_0_L2A  
2020-11-30                S2A_33VUE_20201130_0_L2A  
2020-11-20                S2A_32VPJ_20201120_0_L2A  
2020-11-20                S2A_33VUD_20201120_0_L2A  
2020-11-20                S2A_32VPK_20201120_1_L2A  
2020-11-20                S2A_33VUE_20201120_1_L2A  



In the end, we are printing a summary of the returned data. So as you can see below, we have the image ID and the date taken.

## Catalog Assets
We can now call the asset metadata for the returned items. Let us take a look at the asset of the first item returned. The returned data type is JSON type, so we can use IPython JSON to prettify the result and also have the ability to filter out the data.

In [ ]:
from IPython.display import JSON
JSON(items[0].assets)

<IPython.core.display.JSON object>

In [ ]:
items[0]

S2B_32VPK_20201225_0_L2A

So this is very convenient, and you can look at the asset metadata of any item inside Jupyter Notebook. Note that you can save this in JSON and open it in Jupyter, and it will render it also as JSON.

## NDVI with COGS
You can copy the URL for the COG TIF images and read them with any other Desktop GIS Software. But you can also use the asset method to access specific bands and read them with Rasterio inside Jupyter Notebook. The good thing about this is that you can use the bands' common names to access specific bands and items.

Let us calculate the NDVI values from the Red (B4) and Near-Infrared Band (B8).

In [ ]:
# Read and open(B4 and B8)
b4 = rio.open(items[0].assets(“red”)[“href”])
b8 = rio.open(items[0].assets(“nir”)[“href”])
red = b4.read()
nir = b8.read()
# Calculate ndvi
ndvi = (nir.astype(float)-red.astype(float))/(nir+red)

SyntaxError: ignored

In [ ]:
items[0].assets(“red”)[“href”]

SyntaxError: ignored

In [ ]:
# 从noaa获取数据

In [ ]:
import xarray as xr

In [ ]:
baseURL = 'http://www.esrl.noaa.gov'
catalogURL = '/psd/thredds/dodsC/Datasets/noaa.ersst.v5/sst.mnmean.nc'
sst = xr.open_dataset(baseURL+catalogURL)
print(sst)

<xarray.Dataset>
Dimensions:    (lat: 89, lon: 180, time: 2020, nbnds: 2)
Coordinates:
  * lat        (lat) float32 88.0 86.0 84.0 82.0 ... -82.0 -84.0 -86.0 -88.0
  * lon        (lon) float32 0.0 2.0 4.0 6.0 8.0 ... 352.0 354.0 356.0 358.0
  * time       (time) datetime64[ns] 1854-01-01 1854-02-01 ... 2022-04-01
Dimensions without coordinates: nbnds
Data variables:
    time_bnds  (time, nbnds) float64 ...
    sst        (time, lat, lon) float32 ...
Attributes: (12/38)
    climatology:                     Climatology is based on 1971-2000 SST, X...
    description:                     In situ data: ICOADS2.5 before 2007 and ...
    keywords_vocabulary:             NASA Global Change Master Directory (GCM...
    keywords:                        Earth Science > Oceans > Ocean Temperatu...
    instrument:                      Conventional thermometers
    source_comment:                  SSTs were observed by conventional therm...
    ...                              ...
    license:     

In [ ]:
! wget https://psl.noaa.gov/thredds/catalog/Datasets/COBE/catalog.html?dataset=Datasets/COBE/sst.mon.mean.nc


--2022-05-25 18:18:10--  https://psl.noaa.gov/thredds/catalog/Datasets/COBE/catalog.html?dataset=Datasets/COBE/sst.mon.mean.nc
Resolving psl.noaa.gov (psl.noaa.gov)... 140.172.38.12
Connecting to psl.noaa.gov (psl.noaa.gov)|140.172.38.12|:443... connected.
HTTP request sent, awaiting response... 200 
Length: unspecified [text/html]
Saving to: ‘catalog.html?dataset=Datasets%2FCOBE%2Fsst.mon.mean.nc’

catalog.html?datase     [ <=>                ]   2.42K  --.-KB/s    in 0s      

2022-05-25 18:18:10 (152 MB/s) - ‘catalog.html?dataset=Datasets%2FCOBE%2Fsst.mon.mean.nc’ saved [2474]



In [ ]:
test = xr.open_dataset('https://psl.noaa.gov/thredds/dodsC/Datasets/kaplan_sst/sst.mon.anom.nc')
test

<xarray.Dataset>
Dimensions:  (lon: 72, time: 1996, lat: 36)
Coordinates:
  * lon      (lon) float32 2.5 7.5 12.5 17.5 22.5 ... 342.5 347.5 352.5 357.5
  * time     (time) datetime64[ns] 1856-01-01 1856-02-01 ... 2022-04-01
  * lat      (lat) float32 -87.5 -82.5 -77.5 -72.5 -67.5 ... 72.5 77.5 82.5 87.5
Data variables:
    sst      (time, lat, lon) float32 ...
Attributes:
    title:                           Kaplan Extended SST V2
    platform:                        Analysis
    Conventions:                     COARDS
    history:                         Originally converted from the IRI format...
    dataset_title:                   Kaplan Extended V2 SST anomaly data
    description:                     Data is processed at Lamont-Doherty usin...
    References:                      https://www.psl.noaa.gov/data/gridded/da...
    DODS_EXTRA.Unlimited_Dimension:  time